In [3]:
from __future__ import print_function
import os
import h5py
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from torch.utils.data import Dataset,DataLoader
import torch.nn.init as init

import sklearn.metrics as metrics



In [ ]:
import wandb
#insert here wandb key
WANDB_KEY=""
if WANDB_KEY:
    wandb.login(key=WANDB_KEY)

## Dataset

In [17]:

class MyDataset(Dataset):
    def __init__(self,args,partition='train'):
        self.args=args
        self.dataset_file=h5py.File(self.args["dataset_file_path"], 'r')
        self.num_train=min(args["num_train_samples"],len(self.dataset_file["train_data"]))
        self.num_test=min(args["num_test_samples"],len(self.dataset_file["test_data"]))

        if args["dataset_in_ram"]:
            self.train_set=self.dataset_file["train_data"][:self.num_train]
            self.train_labels=self.dataset_file["train_labels"][:self.num_train]
            self.test_set=self.dataset_file["test_data"][:self.num_test]
            self.test_labels=self.dataset_file["test_labels"][:self.num_test]

        self.partition = partition        

    def __getitem__(self, item):
        if self.partition=="train":
            if self.args["dataset_in_ram"]:
                pointclouds_pair=self.train_set[item]
                label=self.train_labels[item]
            else:
                pointclouds_pair=self.dataset_file["train_data"][item]
                label=self.dataset_file["train_labels"][item]
        else:
            if self.args["dataset_in_ram"]:
                pointclouds_pair=self.test_set[item]
                label=self.test_labels[item]
            else:
                pointclouds_pair=self.dataset_file["test_data"][item]
                label=self.dataset_file["test_labels"][item]
        return pointclouds_pair, label

    def __len__(self):
        if self.partition=="train":
            return self.num_train
        else:
            return self.num_test


## Util

In [12]:

def cal_loss(pred, gold, smoothing=False):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    #we do not use label smoothing

    gold = gold.contiguous().view(-1)

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)

        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.binary_cross_entropy(pred, gold, reduction='mean')

    return loss

#class to print on file
class IOStream():
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text+'\n')
        self.f.flush()

    def close(self):
        self.f.close()


## Model

In [13]:

def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx

#aggiunto parametro device in modo da usare cpu se non gpu
def get_graph_feature(x, k=20,device="cpu", idx=None, dim9=False):
    batch_size = x.size(0)
    num_points = x.size(2)
    #print(x.shape)
    x = x.view(batch_size, -1, num_points)
    #print(x.shape)

    if idx is None:
        if dim9 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, 6:], k=k)
    device = torch.device(device)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
  
    return feature      # (batch_size, 2*num_dims, num_points, k)


In [15]:

class DGCNN_cls(nn.Module):
    def __init__(self, args, output_channels=1):
        super(DGCNN_cls, self).__init__()
        self.args = args
        self.device="cuda" if self.args["cuda"] else "cpu"
        self.k = args["k"]
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args["emb_dims"])

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args["emb_dims"], kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(args["emb_dims"]*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout(p=args["dropout"])
        self.linear2 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dp2 = nn.Dropout(p=args["dropout"])
        self.linear3 = nn.Linear(256, args["final_dim_DGCNN"])

        #final classification layer
        if args["dim_classification"]>0:
            self.beginClassification = nn.Linear(args["final_dim_DGCNN"]*2, args["dim_classification"])
            self.midClassification = nn.Linear(args["dim_classification"], args["dim_classification"])
            self.endClassification = nn.Linear(args["dim_classification"] , output_channels)

        else:
            self.classification= nn.Linear(args["final_dim_DGCNN"]*2, output_channels)


    
    def dgcnn_step(self,x):
        batch_size = x.size(0)
        x = get_graph_feature(x, k=self.k,device=self.device)      # (batch_size, 3, num_points) -> (batch_size, 3*2, num_points, k)
        x = self.conv1(x)                       # (batch_size, 3*2, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k,device=self.device)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv2(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k,device=self.device)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 128, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 128, num_points, k) -> (batch_size, 128, num_points)

        x = get_graph_feature(x3, k=self.k,device=self.device)     # (batch_size, 128, num_points) -> (batch_size, 128*2, num_points, k)
        x = self.conv4(x)                       # (batch_size, 128*2, num_points, k) -> (batch_size, 256, num_points, k)
        x4 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 256, num_points, k) -> (batch_size, 256, num_points)

        x = torch.cat((x1, x2, x3, x4), dim=1)  # (batch_size, 64+64+128+256, num_points)

        x = self.conv5(x)                       # (batch_size, 64+64+128+256, num_points) -> (batch_size, emb_dims, num_points)
        x1 = F.adaptive_max_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x2 = F.adaptive_avg_pool1d(x, 1).view(batch_size, -1)           # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims)
        x = torch.cat((x1, x2), 1)              # (batch_size, emb_dims*2)

        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2) # (batch_size, emb_dims*2) -> (batch_size, 512)
        x = self.dp1(x)
        x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2) # (batch_size, 512) -> (batch_size, 256)
        x = self.dp2(x)
        x = self.linear3(x)                                             # (batch_size, 256) -> (batch_size, output_channels)
        return x

    def forward(self, x):
        #first pointcloud
        first=x[:,0,:,:]
        second=x[:,1,:,:]

        #compute DGCNN output from both pointclouds 
        ret1=self.dgcnn_step(first)      
        ret2=self.dgcnn_step(second)

        #combine output and use classifier
        x=torch.cat((ret1,ret2),dim=1)
        if self.args["dim_classification"]>0:
            x=self.beginClassification(x)
            x=self.midClassification(x)
            x=self.endClassification(x)
        else: 
            x=self.classification(x)

        return x


## Training

In [ ]:

def train(args, io):
    # num_workers=0 to make it work on windows
    train_loader = DataLoader(MyDataset(args,partition='train'), 
            num_workers=args["num_workers"],batch_size=args["batch_size"], shuffle=True, drop_last=True)
    test_loader = DataLoader(MyDataset(args,partition='test'), 
            num_workers=args["num_workers"],batch_size=args["test_batch_size"], shuffle=True, drop_last=False)

    device = torch.device("cuda" if args["cuda"] else "cpu")

    model = DGCNN_cls(args).to(device)

    print("Model loaded!")

    model = nn.DataParallel(model)
    print("Let's use", torch.cuda.device_count(), "GPUs!")

    if args["use_sgd"]:
        print("Use SGD")
        opt = optim.SGD(model.parameters(), lr=args["lr"]*100,
                momentum=args["momentum"], weight_decay=1e-4)
    else:
        print("Use Adam")
        opt = optim.Adam(model.parameters(), lr=args["lr"], weight_decay=1e-4)

    if args["scheduler"] == 'cos':
        scheduler = CosineAnnealingLR(opt, args["epochs"], eta_min=1e-3)
    elif args["scheduler"] == 'step':
        scheduler = StepLR(opt, step_size=20, gamma=0.7)
    
    criterion = cal_loss

    if args["wand_project_name"]:
            wandb.init(
        # Set the project where this run will be logged
        project=args["wand_project_name"], 
        # Track hyperparameters and run metadata
        config=args)




    print(args)
    #
    # TRAINING
    #
    for epoch in range(args["epochs"]):
        train_loss = 0.0
        count = 0.0
        model.train()
        train_pred = []
        train_true = []
        print("Starting epoch ",epoch)
        correct_samples=0
        total_samples=0
        for data, label in train_loader:
            #print("get data")
            data, label = data.to(device), label.to(device).squeeze()
            #print(label)
            data=data.float()
            label=label.float()
            data = data.permute(0,1,3,2)
            batch_size = data.size()[0]
            opt.zero_grad()
            logits = model(data)
            logits=torch.squeeze(torch.sigmoid(logits))
            loss = criterion(logits, label)
            
            loss.backward()
            opt.step()
            
            #preds = logits.max(dim=1)[1] #not needed, we have only one value for each sample
            preds=(logits>0.5).int()
            count += batch_size
            train_loss += loss.item() * batch_size
            train_true.append(label.cpu().numpy())
            for i in range(len(preds)):
                total_samples+=1
                if preds[i]==label[i]:
                    correct_samples+=1
            train_pred.append(preds.detach().cpu().numpy())
        
        if args["scheduler"] == 'cos':
            scheduler.step()
        elif args["scheduler"] == 'step':
            if opt.param_groups[0]['lr'] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]['lr'] < 1e-5:
                for param_group in opt.param_groups:
                    param_group['lr'] = 1e-5

        
        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        print("my accuracy: ", correct_samples/total_samples)
        train_acc=metrics.accuracy_score(train_true, train_pred)
        train_avg_acc= metrics.balanced_accuracy_score(train_true, train_pred)
        wandb.log({"train_acc": train_acc,"train_avg_acc": train_avg_acc, "train_loss": loss})
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,train_loss*1.0/count,train_acc,train_avg_acc)
        io.cprint(outstr)

        ####################
        # Test
        ####################
        test_loss = 0.0
        count = 0.0
        model.eval()
        test_pred = []
        test_true = []
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            data=data.float()
            label=label.float()

            data = data.permute(0,1,3,2)
            batch_size = data.size()[0]
            logits = model(data)
            logits=torch.squeeze(torch.sigmoid(logits))
            loss = criterion(logits, label)
            
            #preds = logits.max(dim=1)[1] #not needed, we have only one value for each sample
            preds=(logits>0.5).int()

            count += batch_size
            test_loss += loss.item() * batch_size
            test_true.append(label.cpu().numpy())
            test_pred.append(preds.detach().cpu().numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        test_acc = metrics.accuracy_score(test_true, test_pred)
        test_avg_acc = metrics.balanced_accuracy_score(test_true, test_pred)
        wandb.log({"test_acc": test_acc,"test_avg_acc": test_avg_acc, "test_loss": loss})
        outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,
                                                                              test_loss*1.0/count,
                                                                              test_acc,
                                                                              test_avg_acc)
        io.cprint(outstr)
        #end epoch
    wandb.finish()



In [18]:

def test(args, io):
    test_loader = DataLoader(MyDataset(dataset_file_name=args["dataset_file_path"],partition='test',num_points=args["num_points"]), 
            num_workers=args["num_workers"],batch_size=args["test_batch_size"], shuffle=True, drop_last=False)

    device = torch.device("cuda" if args["cuda"] else "cpu")

    model = DGCNN_cls(args).to(device)

    model = nn.DataParallel(model)
    
    model.load_state_dict(torch.load(args["model_path"]))
    model = model.eval()
    test_acc = 0.0
    count = 0.0
    test_true = []
    test_pred = []
    for data, label in test_loader:

        data, label = data.to(device), label.to(device).squeeze()
        data = data.permute(0, 2, 1)
        batch_size = data.size()[0]
        logits = model(data)
        #preds = logits.max(dim=1)[1]
        preds=(logits>0.5).int()
        test_true.append(label.cpu().numpy())
        test_pred.append(preds.detach().cpu().numpy())
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    test_acc = metrics.accuracy_score(test_true, test_pred)
    avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
    outstr = 'Test :: test acc: %.6f, test avg acc: %.6f'%(test_acc, avg_per_class_acc)
    io.cprint(outstr)



## Main

In [19]:

def _init_():
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    if not os.path.exists('outputs/'+args["exp_name"]):
        os.makedirs('outputs/'+args["exp_name"])
    if not os.path.exists('outputs/'+args["exp_name"]+'/'+'models'):
        os.makedirs('outputs/'+args["exp_name"]+'/'+'models')

if __name__ == "__main__":
    
    args={
        "exp_name": "exp",
        "wand_project_name": "Fragment_EAI",
        #dataset
        "dataset_file_path": "dataset_90810pairs_100points.h5py",
        "dataset_in_ram": True,     # true:load dataset in ram, false: get each elem from dataset file
        "num_train_samples": 15000,     # Num of pairs to consider for train
        "num_test_samples": 5000,       # Num of pairs to consider for test
        "batch_size": 32,           # Size of batch
        "test_batch_size": 16,      # Size of batch
        "num_workers": 0,           # parallel workers on dataloader, 0 if windows

        #model
        "architecture": "DGCNN",
        "num_points": 100,          # num of points to use
        "dropout": 0.5,             # initial dropout rate
        "emb_dims": 1024,           # Dimension of embeddings
        "k": 20,                    # Num of nearest neighbors to use
        "final_dim_DGCNN": 64,      #dimension of final linear layer of DGCNN, max=256
        "dim_classification": 64,   #dim of mid final classification layer, max= final_dim_DGCNN*2
                                    #if 0 no middle layer

        #training
        "epochs": 50,               # number of episode to train
        "use_sgd": False,           # Use SGD
        "lr": 0.001,                # learning rate (default: 0.001, 0.1 if using sgd)
        "momentum": 0.9,            # SGD momentum (default: 0.9)
        "scheduler": "cos",         # Scheduler to use, [cos, step]
        "seed": 1,                  # random seed (default: 1)
        "eval": False,              # evaluate the model

    }
    
    _init_()

    io = IOStream('outputs/' + args["exp_name"] + '/run.log')
    io.cprint(str(args))
    
    #set up GPU
    args["cuda"] = torch.cuda.is_available()
    torch.manual_seed(args["seed"])
    if args["cuda"]:
        io.cprint(
            'Using GPU : ' + str(torch.cuda.current_device()) + ' from ' + str(torch.cuda.device_count()) + ' devices')
        torch.cuda.manual_seed(args["seed"])
    else:
        io.cprint('Using CPU')

    if not args["eval"]:
        train(args, io)
    else:
        test(args, io)


{'exp_name': 'exp', 'model': 'dgcnn', 'dataset': 'modelnet40', 'batch_size': 16, 'test_batch_size': 16, 'epochs': 50, 'use_sgd': False, 'lr': 0.001, 'momentum': 0.9, 'scheduler': 'cos', 'no_cuda': False, 'seed': 1, 'eval': False, 'num_points': 100, 'dropout': 0.5, 'emb_dims': 1024, 'k': 20, 'model_path': '', 'max_elements': 30}
Using CPU
Creating dataset...
Each point cloud is sampled with 100 points
maximum number of objects reached, finishing dataset
Dataset contains 51 fragments--> 435 pairs 
Creating dataset...
Each point cloud is sampled with 100 points
maximum number of objects reached, finishing dataset
Dataset contains 51 fragments--> 435 pairs 
Model loaded!
Let's use 0 GPUs!
Use Adam
{'exp_name': 'exp', 'model': 'dgcnn', 'dataset': 'modelnet40', 'batch_size': 16, 'test_batch_size': 16, 'epochs': 50, 'use_sgd': False, 'lr': 0.001, 'momentum': 0.9, 'scheduler': 'cos', 'no_cuda': False, 'seed': 1, 'eval': False, 'num_points': 100, 'dropout': 0.5, 'emb_dims': 1024, 'k': 20, 'mode

C:\Users\alessio\Anaconda3\envs\EAI\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train 0, loss: 0.620520, train acc: 0.648148, train avg acc: 0.500000
Test 0, loss: 0.589304, test acc: 0.648276, test avg acc: 0.500000
Starting epoch  1
Train 1, loss: 0.587883, train acc: 0.648148, train avg acc: 0.500000


KeyboardInterrupt: 